# Sentinel-2l2a with no clouds

In [19]:
import numpy as np
import rasterio
from pathlib import Path
from rasterio.enums import Resampling

In [21]:
# Set paths
input_dir = Path("G:/Semester4/Innolab/datacube_rwanda/data/sentinel2_l2a")
output_dir = Path("G:/Semester4/Innolab/datacube_rwanda/data/sentinel2_l2a_cloud_masked")

## Create cloud mask from SCL band

In [24]:
def get_cloud_mask(scl_band):
    valid_mask = ~np.isin(scl_band, [1, 3, 8, 9, 10])  # Keep non-cloudy pixels
    return valid_mask


## Resample SCL band to match the shape of the target band

In [27]:

def resample_scl(scl_path, target_shape):
    with rasterio.open(scl_path) as scl_src:
        scl_data = scl_src.read(1, out_shape=target_shape, resampling=Resampling.nearest)
    return scl_data

## Apply cloud masking to all bands for a specific scene

In [30]:
def mask_clouds_for_scene(scene_dir, output_dir):
    date_str = scene_dir.name
    masked_dir = output_dir / date_str
    masked_dir.mkdir(parents=True, exist_ok=True)

    scl_files = list(scene_dir.glob("*SCL*.tif"))
    if not scl_files:
        print(f"SCL band not found for {date_str}, skipping...")
        return None

    scl_path = scl_files[0]
    for band_file in scene_dir.glob("B*.tif"):  # Match B01.tif, B02.tif, etc.
        if 'SCL' in band_file.name:
            continue

        with rasterio.open(band_file) as src:
            data = src.read(1)
            profile = src.profile.copy()
            target_shape = (1, data.shape[0], data.shape[1])
            
            scl_resampled = resample_scl(scl_path, target_shape[1:])
            cloud_mask = get_cloud_mask(scl_resampled)
            
            masked_data = np.where(cloud_mask, data, profile['nodata'])
            output_path = masked_dir / band_file.name
            
            with rasterio.open(output_path, 'w', **profile) as dst:
                dst.write(masked_data, 1)
    
    print(f"Cloud masking applied for {date_str}")
    return masked_dir

## Process all scenes in the input directory

In [33]:
def process_all_scenes(input_dir, output_dir):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    
    masked_dirs = []
    for scene_dir in input_dir.iterdir():
        if scene_dir.is_dir():
            result = mask_clouds_for_scene(scene_dir, output_dir)
            if result:
                masked_dirs.append(result)
    return masked_dirs

In [35]:
# Run processing
process_all_scenes(input_dir, output_dir)

SCL band not found for S2A_MSIL2A_20240531T080611_R078_T35MQU_20240531T150446, skipping...
SCL band not found for S2A_MSIL2A_20240610T080611_R078_T35MRU_20240610T142004, skipping...
SCL band not found for S2A_MSIL2A_20240620T080611_R078_T35MQU_20240620T135853, skipping...
SCL band not found for S2A_MSIL2A_20240620T080611_R078_T35MRU_20240620T135115, skipping...
SCL band not found for S2A_MSIL2A_20240630T080611_R078_T35MQU_20240701T185708, skipping...
SCL band not found for S2A_MSIL2A_20240630T080611_R078_T35MRU_20240701T190110, skipping...
SCL band not found for S2A_MSIL2A_20240630T080611_R078_T35MRU_20240701T190926, skipping...
Cloud masking applied for S2A_MSIL2A_20240819T080611_R078_T35MQU_20240819T130750
SCL band not found for S2B_MSIL2A_20240127T081119_R078_T35MRU_20240127T120827, skipping...
SCL band not found for S2B_MSIL2A_20240307T080759_R078_T35MRU_20240307T123828, skipping...
SCL band not found for S2B_MSIL2A_20240317T080649_R078_T35MQU_20240317T121450, skipping...
SCL band 

[WindowsPath('G:/Semester4/Innolab/datacube_rwanda/data/sentinel2_l2a_cloud_masked/S2A_MSIL2A_20240819T080611_R078_T35MQU_20240819T130750'),
 WindowsPath('G:/Semester4/Innolab/datacube_rwanda/data/sentinel2_l2a_cloud_masked/S2B_MSIL2A_20240705T080609_R078_T35MRU_20240706T024811'),
 WindowsPath('G:/Semester4/Innolab/datacube_rwanda/data/sentinel2_l2a_cloud_masked/S2B_MSIL2A_20240715T080609_R078_T35MRU_20240715T131113'),
 WindowsPath('G:/Semester4/Innolab/datacube_rwanda/data/sentinel2_l2a_cloud_masked/S2B_MSIL2A_20240725T080609_R078_T35MQU_20240725T102837'),
 WindowsPath('G:/Semester4/Innolab/datacube_rwanda/data/sentinel2_l2a_cloud_masked/S2B_MSIL2A_20240725T080609_R078_T35MRU_20240725T102837'),
 WindowsPath('G:/Semester4/Innolab/datacube_rwanda/data/sentinel2_l2a_cloud_masked/S2B_MSIL2A_20240804T080609_R078_T35MQU_20240804T111459'),
 WindowsPath('G:/Semester4/Innolab/datacube_rwanda/data/sentinel2_l2a_cloud_masked/S2B_MSIL2A_20240804T080609_R078_T35MRU_20240804T111459'),
 WindowsPath(